<p align="center">
    <img src="./img/image-1.png">
<p>

# Éxito en videojuegos

La industria de los videojuegos se ha convertido en los ultimos 10 años, en uno de los sectores con mayor recaudación del mundo, más que el cine y la música juntos. Estamos hablando de un sector que generó en el 2023 alrededor de $184.0 billones de dolares.

Sin embargo, uno de los puntos que el público general puede no tomar en cuenta son los costes generales de desarrollo. Para hacernos una idea de esto, a comienzos de la sexta generación de consolas (Xbox, PS2 y GameCube) los costes de desarrollo rondaban entre los $2-$7 millones de dolares para juegos medianos, y entre $15-$60 millones para juegos AAA. el abánico es bastante amplio, pero esto dependía de cuan ambiciosos fuesen los estudios y publishers en sus lanzamientos, años de desarrollo, marketing, licencias si eran multiplataforma, etc. Hoy, en la novena generación de consolas, los costes de desarrollo comprenden rangos muy elevados: desde $20-$50 millones de dolares para juegos medianos, y desde $60-$600 millones de dolares para juegos AAA. Este rango tan disperso comprenden juegos de todo tipo, así como grandes y nuevas IPs: Final Fantasy XVI ($60 millones, 87 MC, 2023), GTA 5 ($500 millones, 97 MC, 2013), Star Citizen ($600 millones, en Acceso anticipado).

<sup>[Aquí puedes ver la lista de los juegos más caros de la historia](https://en.wikipedia.org/wiki/List_of_most_expensive_video_games_to_develop)</sup>

El problema de todo esto vienen siendo los benificios que se esperan de estos juegos. Aunque hay mucho caso de exito, hay otros los cuales no han corrido la misma suerte: SONY lanzó en el mes de agosto Concord (62 MC), un juego multijugador de 5V5 que se estima costó $400 millones, vendiendo 25000 unidades y una media de 700 jugadores, [muy lejos de cubrir un porcentaje minúsculo del desarrollo y siendo un absoluto fracaso,](https://jonahwrites.blog/2024/09/05/concord-the-biggest-flop-in-gaming-history/) tanto que cerraron los servidores pasadas dos semanas de su lanzamiento y devolviendo el dinero a los jugadores. 

Tambien estamos hablando de un caso muy evidente pero, ¿y si te dijera que incluso juegos muy bien valorados tampoco se salvan? Alan Wake 2 (89 MC) tuvó un coste de desarrollo de $70 millones de dolares y siendo muy bien recibido por la comunidad y critica. Sin embargo, [vendió 1.3m según un reporte de Remedy en febrero de 2024,](https://www.eurogamer.net/alan-wake-2-fastest-selling-remedy-game-but-yet-to-turn-a-profit) lo cual no llega a cubrir los costes de desarrollo. Este caso es parecido al de Tango Gameworks, pero con el cierre del estudio por parte de Microsoft debido a no cumplir expectativas de ventas.

¿Tiene algo que ver el género o la nota recibida? ¿Los jugadores a los que el juego va dirigido? ¿Quizá sea un tema de timing? ¿El género más popular suele ser el más rentable? Todas estas dudas y más las estaremos afrontando a lo largo de este analisis, en el que queremos abarcar estas dudas gracias a los datos.

## Nuestras hipótesis
+ ¿Un videojuego exitoso en cuanto a critica representa el éxito en cuanto a ventas?
+ ¿La base de jugadores de un videojuego garantiza el éxito?
+ ¿Importa el género de un videojuego?
+ ¿La fecha de lanzamiento de un videojuego es un elemento clave para máximizar ingresos?
+ ¿Se pueden tener en cuenta otros parámetros para medir el éxito? 

<sup><sub>
*Este análisis toma en cuenta el mercado del PC **usando datasets de Steam:*** <br>
[Steam Games Dataset](https://www.kaggle.com/datasets/fronkongames/steam-games-dataset) <br>
[2024 Steam Statistics | Python | SQL | Tableau](https://www.kaggle.com/code/johnangelobelarma/2024-steam-statistics-python-sql-tableau/input)
</sub></sup>

## OBTENCIÓN DE LOS DATOS

In [1]:
import pandas as pd
import numpy as np
import time
#pd.set_option('display.max_columns', 200)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

In [37]:
# Pasos para limpiar el dataset original, he creado uno nuevo que pesará menos y poder subirlo a GitHub

#1st CSV
games_df_column_names = ['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU', 
                    'Required age', 'Price', 'Unknown', 'DiscountDLC count', 'About the game', 
                    'Supported languages', 'Full audio languages', 'Reviews', 'Header image', 
                    'Website', 'Support url', 'Support email', 'Windows', 'Mac', 
                    'Linux', 'Metacritic score', 'Metacritic url', 'User score', 
                    'Positive', 'Negative', 'Score rank', 'Achievements', 
                    'Recommendations', 'Notes', 'Average playtime forever', 
                    'Average playtime two weeks', 'Median playtime forever', 
                    'Median playtime two weeks', 'Developers', 'Publishers', 
                    'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies']

df_1 = pd.read_csv("./../data/games.csv", header=None, skiprows=1)
df_1.columns = games_df_column_names

columns_todrop = ["Estimated owners", "Peak CCU", "Required age", "Unknown", "About the game", "Supported languages",
                  "Full audio languages", "Reviews", "Header image", "Website", "Support url", "Support email", "Windows", "Mac",
                  "Linux", "Metacritic url", "Positive", "Negative", "Score rank", "Achievements", "Recommendations", "Notes", "Tags",
                  "Screenshots", "Movies", "Average playtime two weeks" ,"Median playtime forever", "Median playtime two weeks",
                  "DiscountDLC count", "User score"]
df_1.drop(columns_todrop, axis=1, inplace=True)

# Inserting new colums
df_1.insert(9, "Category_1", np.nan)
df_1.insert(10, "Category_2", np.nan)
df_1.insert(12, "Genre_1", np.nan)
df_1.insert(13, "Genre_2", np.nan)

# Parsing and resetting index
df_1[["Category_1", "Category_2", "Genre_1", "Genre_2"]] = df_1[["Category_1", "Category_2", "Genre_1", "Genre_2"]].astype(object)
df_1["Categories"] = df_1["Categories"].astype(str)
df_1.reset_index(drop=True, inplace=True)

# Categories split
for i, categories in enumerate(df_1.copy()["Categories"].str.split(",")):
    df_1.iloc[i,9] = categories[0]
    if len(categories) > 1:
        df_1.iloc[i,10] = categories[1]
    else:
        continue

# Date Formatting
for i,date_string in enumerate(df_1["Release date"]):
    date = pd.to_datetime(date_string)
    # Convertir al formato deseado (DD-MM-YYYY)
    formatted_date = date.strftime('%d-%m-%Y')
    df_1.iloc[i,2] =  formatted_date

# Genres split
for i, genres in enumerate(df_1.copy()["Genres"].str.split(",")):
    if df_1.iloc[i,11] is not np.nan:
        df_1.iloc[i,12] = genres[0]
        if len(genres) > 1:
            df_1.iloc[i,13] = genres[1]
    else:
        continue

# Step 4 Specify the file path to save data
csv_file_path = "games-xs.csv"
df_1.to_csv(csv_file_path)

In [49]:
df_1 = df_1 = pd.read_csv("./../data/games-xs.csv")
df_1.drop(columns=["Unnamed: 0","Name", "Release date", "Price", "Metacritic score", "Average playtime forever", "Developers",
                   "Publishers"], inplace=True)

In [50]:
# 2nd CSV
df_2 = pd.read_csv("./../data/Steam_2024_bestRevenue_1500.csv")
df_2.rename(columns={"steamId":"AppID", "developers":"Developers", "publishers":"Publishers", 
             "publisherClass":"PublisherClass", "revenue":"Revenue", 
             "price":"Price", "ReleaseDate":"Release date", "name":"Name"}, inplace=True)

In [51]:
games = df_2.merge(df_1, on=["AppID"], how="left")

In [82]:
df_2

,Name,releaseDate,copiesSold,Price,Revenue,avgPlaytime,reviewScore,PublisherClass,Publishers,Developers,AppID
0,WWE 2K24,07-03-2024,165301,99.99,8055097.0,42.365140,71,AAA,2K,Visual Concepts,2315690
1,EARTH DEFENSE FORCE 6,25-07-2024,159806,59.99,7882151.0,29.651061,57,Indie,D3PUBLISHER,SANDLOT,2291060
2,Sins of a Solar Empire II,15-08-2024,214192,49.99,7815247.0,12.452593,88,Indie,Stardock Entertainment,"Ironclad Games Corporation,Stardock Entertainment",1575940
3,Legend of Mortal,14-06-2024,440998,19.99,7756399.0,24.797817,76,Indie,"Paras Games,Obb Studio Inc.",Obb Studio Inc.,1859910
4,Shin Megami Tensei V: Vengeance,13-06-2024,141306,59.99,7629252.0,34.258496,96,AA,SEGA,ATLUS,1875830
...,...,...,...,...,...,...,...,...,...,...,...
1495,Infection Free Zone,11-04-2024,452146,24.99,8739530.0,14.494432,73,Indie,Games Operators,Jutsu Games,1465460
1496,Dark and Darker,07-06-2024,2640903,0.00,8706135.0,31.974027,74,Indie,IRONMACE,IRONMACE,2016590
1497,Songs of Conquest,20-05-2024,501474,34.99,8641459.0,11.891152,88,AA,Coffee Stain Publishing,Lavapotion,867210
1498,GUNDAM BREAKER 4,28-08-2024,156339,59.99,8440898.0,14.733899,81,AA,Bandai Namco Entertainment Inc.,"CRAFTS ＆ MEISTER Co., Ltd",1672500


In [104]:
games[games["Price"] == 0]

,Name,releaseDate,copiesSold,Price,Revenue,avgPlaytime,reviewScore,PublisherClass,Publishers,Developers,AppID,Categories,Category_1,Category_2,Genres,Genre_1,Genre_2
22,Pixel Gun 3D: PC Edition,02-04-2024,1626877,0.0,4.707782e+06,11.897823,62,Indie,Cubic Games Studio,Cubic Games Studio,2524890,"Single-player,Multi-player,PvP,Online PvP,Co-o...",Single-player,Multi-player,"Action,Adventure,Massively Multiplayer,Free to...",Action,Adventure
26,Predecessor,20-08-2024,433605,0.0,4.523840e+06,109.913828,79,Indie,Omeda Studios,Omeda Studios,961200,"Multi-player,PvP,Online PvP,Full controller su...",Multi-player,PvP,"Action,Massively Multiplayer,Strategy,Early Ac...",Action,Massively Multiplayer
27,Palia,25-03-2024,881082,0.0,4.263470e+06,47.027915,78,Indie,Singularity 6 Corporation,Singularity 6 Corporation,2707930,"Single-player,Multi-player,Co-op,Online Co-op,...",Single-player,Multi-player,"Adventure,RPG,Simulation",Adventure,RPG
56,Viking Rise: Valhalla,21-05-2024,20537,0.0,2.595130e+05,36.620297,75,Indie,IGG SINGAPORE PTE. LTD.,IGG SINGAPORE PTE. LTD.,2819520,"Multi-player,MMO,PvP,Online PvP,Co-op,Online C...",Multi-player,MMO,"Massively Multiplayer,RPG,Strategy,Free to Play",Massively Multiplayer,RPG
76,Idle Hero TD - Tower Defense,03-05-2024,13061,0.0,2.358630e+05,189.057011,79,Indie,Swell Games LLC,Swell Games LLC,2897580,"Single-player,Steam Achievements,In-App Purchases",Single-player,Steam Achievements,"Action,Casual,Indie,Strategy,Free to Play,Earl...",Action,Casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1466,Once Human,09-07-2024,5709407,0.0,3.453056e+07,42.261045,72,Indie,Starry Studio,Starry Studio,2139460,"Multi-player,MMO,PvP,Online PvP,Co-op,Online C...",Multi-player,MMO,"Action,Adventure,RPG,Simulation,Strategy,Free ...",Action,Adventure
1479,Firestone: Online Idle RPG,01-04-2024,335093,0.0,1.568936e+07,296.332852,79,Indie,Holyday Studios,Holyday Studios,1013320,"Single-player,Multi-player,PvP,Online PvP,Co-o...",Single-player,Multi-player,"Action,Adventure,Casual,Free to Play,Indie,RPG...",Action,Adventure
1483,Summoners War,07-01-2024,540801,0.0,1.078734e+07,112.856340,0,Indie,Com2uS,Com2uS,2426960,"Multi-player,PvP,Online PvP,In-App Purchases,P...",Multi-player,PvP,"RPG,Strategy,Free to Play",RPG,Strategy
1489,Lust Goddess,21-02-2024,456707,0.0,9.695496e+06,33.544343,48,Indie,BRAWEA LTD,BRAWEA LTD,2808930,"Single-player,Multi-player,PvP,Steam Achieveme...",Single-player,Multi-player,"Action,Adventure,Massively Multiplayer,RPG,Str...",Action,Adventure


# Cosas que interesa buscar para el dataset mediante scrapping
+ Metacritic score: faltan más de 1000 registros, relevantes para la primera hipótesis.<br>
[Página para conseguir esto](https://www.metacritic.com)
+ User Score: Para contrastar la critica de los medios especializados con la de los usuarios. <br>
[Página para conseguir esto](https://steamdb.info/charts/)
+ Copies Sold: Podriamos hacer un análisis estadístico cerrado con los que ya tenemos, pero no sería fiable. <br>
[Página para conseguir esto](https://github.com/molatosekgobela/Data-Science-Video-games-sales-dataset?tab=readme-ov-file)
+ avgPlaytime. <br>
[Página para conseguir esto](https://steamdb.info/charts/) 
+ Publisher Class. <br>
[Página para conseguir esto](https://vginsights.com/publishers-database)